<a href="https://colab.research.google.com/github/anupstar100/UML-Capston_Project-Customer_Segmentation/blob/main/UML_Capston_Project_Online_Retail_Customer_Segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <b><u> Project Title : Extraction/identification of major topics & themes discussed in news articles. </u></b>

## <b> Problem Description </b>

### In this project, your task is to identify major customer segments on a transnational data set which contains all the transactions occurring between 01/12/2010 and 09/12/2011 for a UK-based and registered non-store online retail.The company mainly sells unique all-occasion gifts. Many customers of the company are wholesalers.

## <b> Data Description </b>

### <b>Attribute Information: </b>

* ### InvoiceNo: Invoice number. Nominal, a 6-digit integral number uniquely assigned to each transaction. If this code starts with letter 'c', it indicates a cancellation.
* ### StockCode: Product (item) code. Nominal, a 5-digit integral number uniquely assigned to each distinct product.
* ### Description: Product (item) name. Nominal.
* ### Quantity: The quantities of each product (item) per transaction. Numeric.
* ### InvoiceDate: Invice Date and time. Numeric, the day and time when each transaction was generated.
* ### UnitPrice: Unit price. Numeric, Product price per unit in sterling.
* ### CustomerID: Customer number. Nominal, a 5-digit integral number uniquely assigned to each customer.
* ### Country: Country name. Nominal, the name of the country where each customer resides.

In [ ]:
# MOUNTING THE DRIVE
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# IMPORTING THE REQUIRED LIBRARIES
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

from sklearn.cluster import KMeans

In [ ]:
# LOADING OUR DATA
df = pd.read_csv('/content/drive/MyDrive/Capston Project/Online Retail Customer Segmentation/Online Retail (1).csv', encoding = "ISO-8859-1")

### Glimpses of our data

In [ ]:
# FIRST FIVE ROWS
df.head()

In [ ]:
# LAST FIVE ROWS
df.tail()

In [ ]:
# RANDOM FIVE ROWS
df.sample(5)

In [2]:
# CREATING A COPY OF OUR DATA
df_copy = df.copy()

NameError: ignored

# Data Information

In [ ]:
df.shape

In [ ]:
df.info()

**Observation:**
1. There are 8 columns with 5,41,909 rows of data.
2. There are 5 categorical columns and 3 numerical columns. 

In [ ]:
df.describe()

# Data Cleaning

In [ ]:
# checking for duplicate values
df.duplicated().sum()

In [ ]:
# dropping the duplicates
df.drop_duplicates(inplace = True)

In [ ]:
df.shape

**Observations:**
1. There are 5,268 duplicate rows.
2. Shape of data before dropping the duplicates ---> (541909, 8)
3. Shape of data after dropping the duplicates ---> (536641, 8)

In [ ]:
# NUMBER OF UNIQUE VALUES IN EAH COLUMN
df.nunique()

In [ ]:
# CHECKING FOR NULL VALUES
pd.DataFrame({'Columns' : df.columns,
              'Toatl Nos of Null values' : df.isna().sum(),
              '% of null values' : round(df.isna().mean() * 100,2)}).reset_index().drop(['index'], axis = 1)

**Observations:**
1. There are 1454 null values (0.27%) in `Description` columns.
2. There are 135037 null values (25.16%) in `CustomerID` columns.

In [ ]:
# Check if InvoiceNo for Null Customer ID exist in cases where Customer ID is present for filling CustomerID Nulls
df[df['CustomerID'] == 'NaN']

**Observation:**
* No such cases present as empty dataframe is returned.

In [ ]:
# CREATING A LIST OF UNIQUE INVOICES WHERE CUSTOMER ID IS NULL
null_id_invoices = df[df.CustomerID.isna()]['InvoiceNo'].drop_duplicates().tolist()
print('Invoices count with null Customer ID:  ', len(null_id_invoices))

In [ ]:
# CHECK IF INVOICE NUMBER IN NULL CUSTOMER ID DF EXIST IN NON - NULL CUSTOMER ID DF
df[~df['CustomerID'].isna()][df['InvoiceNo'].isin(null_id_invoices)]

Since the customer ID's are missing, I assume these orders were not made by the customers already in the data set because those customers already have ID's. I also don't want to assign these orders to those customers because this would alter the insights I draw from the data. Instead of dropping the null CustomerID values which amounts to ~25% of data, let's assign those rows a unique customer ID per order using InvoiceNo. This will act as a new customer for each unique order.

In [ ]:
# CHECK IF INVOICE NUMBER HAS UNIQUE MAPPING WITH CUSTOMER ID SO THAT
# EACH INVOICE NUMBER CORRESPONDING TO NULL CUSTOMER ID CAN BE ASSIGN A NEW CUSTOMER.
df.groupby(['InvoiceNo'])['CustomerID'] \
                             .nunique() \
                             .reset_index(name='nunique') \
                             .sort_values(['nunique'], ascending=False) \
                             .head(10)

**Observation:**
* On sorting, this data shows that each invoice related to maximum 1 customer.

In [ ]:
# CREATING NewId COLUMN AND ASSIGNING TO InvoiceNo WHERE CustomerID IS NULL
df['NewID'] = df['CustomerID']
df.loc[df['CustomerID'].isnull(), ['NewID']] = df['InvoiceNo']

# REMOVE ALL NON DIGIT CHARATERS FROM NewID COLUMNS
# SINCE INVOICE CAN CONTAIN 'C' REFERRING TO CANCELLATIONS
df['NewID'] = df['NewID'].astype(str).str.replace('\D+', '')

# CONVERT TO INTEGER
df['NewID'] = pd.to_numeric(df['NewID'])

# CHECK IF PRESENT CustomerIDs AND NewIDs HAVE ANY COMMON VALUES SINCE IT WOULD CREATE ALTER ACTUAL CUSTOMER INSIGHTS
customer = df['CustomerID'].nunique()
null_invoices = df[df.CustomerID.isnull()]['InvoiceNo'].nunique()
new_ids = df['NewID'].nunique()
print("Number of Customers:", customer)
print("Number of Orders where CustomerID in Null:", null_invoices)
print("Number of Customers + Number of Orders where CustomerID in Null:", customer + null_invoices)
print("Number of New ID's:", new_ids)

* Since both values equal, we know all the different orders that didn't have a customer ID got assigned unique NewID and no duplicates were created.

In [ ]:
# RANGE OF InvoiceDate COLUMN
print('Maximum Invoice Date: ', max(df['InvoiceDate']))
print('Minimum Invoice Date: ', min(df['InvoiceDate']))

In [ ]:
# ADDING CANCELLATIONS COLUMN BASED ON DEFINITION THAT InvoiceNo START WITH 'C'
df["cancellations"] = np.where(df["InvoiceNo"].str.startswith('C'), 1, 0)
total_data = df["InvoiceNo"].shape[0]
cancelled_data = df[df.cancellations == 1].shape[0]
print("Number of cancelled products data", cancelled_data, cancelled_data*100/total_data, "\n")

print(df[df.cancellations == 1]["Quantity"].describe())

# REMOVING CANCELLATIONS SINCE THEY HAVE NEGATIVE QUANTITIES AND MAKE ONLY ~2% OF DATA
df = df[df.cancellations == 0]

# EDA

### Analysis of Unit Price Variable

In [ ]:
# MAXIMUM AND MINIMUM UNIT PRICE
print("Minimum UnitPrice", min(df["UnitPrice"]))
print("Maximum UnitPrice", max(df["UnitPrice"]))
print('=== ' * 30 + '\n')

# DESCRIPTION OF UNIT PRICE COLUMN
df["UnitPrice"].describe()

In [ ]:
# ROWS WHERE UNIT PRICE IS LESS THAN ZERO OR NEGATIVE
df[df.UnitPrice < 0]

In [ ]:
# REMOVING THESE ROWS --- THEY ARE TOTAL 2 ROWS. SO IT WILL NOT IMPACT OUR DATA
df = df[df['UnitPrice'] >= 0]

In [ ]:
df.shape

* Afetr removing the rows, the shape of our data becomws (527388, 10)

In [ ]:
# % OF DATA WITH UNIT PRICE = 0
print("% of data with Unit Price = 0: ", round(len(df[df.UnitPrice == 0]) * 100 / len(df),2),"%" )

# NUMBER OF UNIQUE CUSTOMER WITH UNIT PRICE = 0
print("Count of unique Customer ID where Unit Price = 0: ", df[df.UnitPrice == 0].CustomerID.nunique(), "\n")

# GETTING THE DATA WHERE UNIT PRICE IS ZERO BUT CUSTOMER ID IS NOT NULL
print('=== ' * 25)
df[df.UnitPrice == 0][~df.CustomerID.isnull()].head()

In [ ]:
# DROPPING ROWS WITH UNIT PRICE = 0
df = df[df.UnitPrice > 0]
df["UnitPrice"].describe()

In [ ]:
# MAXIMUM AND MUNIMUM QUANTITIES
print("Minimum Quantity", min(df["Quantity"]))
print("Maximum Quantity", max(df["Quantity"]))
print('=== ' * 25 + '\n')

# STATISTICAL VALUES OF QUANTITY COLUMN
df["Quantity"].describe()

### Customer and total revenue split wrto country

In [ ]:
# ADDING MONETARY INFORMATION BY CALCULATING TOTAL VALUE OF TRANSACTION
df["Total_sales"] = df["UnitPrice"] * df["Quantity"]

# CREATING COUNTRY LEVEL GROUPING TO FIND UNIQUE CUSTOMER COUND AND %
country_customer_df = df.groupby("Country")["CustomerID"].nunique().reset_index().rename(columns = \
                                                                                           {"CustomerID":"count_CustomerID"})
country_customer_df["customer_%"] = round(country_customer_df["count_CustomerID"] * \
                                          100/country_customer_df["count_CustomerID"].sum(),2)

# CREATING COUNTRY LEVEL GROUPING TO FIND TOTAL REVENUE AND %
country_df = df.groupby("Country")["Total_sales"].sum().reset_index()
country_df["Total_sales%"] = round(country_df["Total_sales"] * 100 / country_df["Total_sales"].sum(),2)

In [ ]:
# COUNTRY V/S CUSTOMER %
country_customer_df = country_customer_df.sort_values(by = "customer_%", ascending = False)
fig, ax = plt.subplots(figsize = (10, 4), dpi = 100)
ax = sns.barplot(x = country_customer_df["Country"], y = country_customer_df['customer_%'])
ax.set_xticklabels(ax.get_xticklabels(), rotation = 50, ha = "right")
plt.show()

# COUNTRY V/S TOTAL SALES %
country_df = country_df.sort_values(by = "Total_sales%", ascending = False)
fig, ax = plt.subplots(figsize = (10, 4), dpi = 100)
ax = sns.barplot(x = country_df["Country"], y = country_df['Total_sales%'])
ax.set_xticklabels(ax.get_xticklabels(), rotation = 50, ha = "right")
plt.show()

* The United Kingdom not only has the most sales revenue, but also the most customers. Therefore, for the purpose of this analysis, I will be taking data corresponding to orders from the United Kingdom.

In [ ]:
# CREATING A UK DF
uk_df = df[df.Country == "United Kingdom"]
uk_df.info()

In [ ]:
# CHECKING MOST POPULAR PRODUCT IN UK
uk_product = uk_df.groupby(['StockCode', 'Description'], as_index= False)['Quantity'].sum().sort_values(by = 'Quantity', ascending = False)
uk_product.head(5)

In [ ]:
# FIND THE UNIQUE NUMBER OF InvoiceNo PER CUSTOMER FOR ANALYZING UK 
groupby_customers = pd.DataFrame(uk_df.groupby('NewID')['InvoiceNo'].nunique())
groupby_customers.describe()

In [ ]:
# FIND THE UNIQUE MBER OF PRODUCTS PER ORDER
groupby_invoice = pd.DataFrame(uk_df.groupby('InvoiceNo')['StockCode'].nunique())
groupby_invoice.columns = ['Number of products per Order']
groupby_invoice.describe()

* The average number of orders per customer is 1 and average number of products per Order is 15

# RFM Analysis

RFM is a data modeling method used to analyze customer value. It stands for Recency, Frequency and Monetary, which are just three metrics that describe what your customers did.

* *Recency (R) of a customer* - Days since the last purchase.
* *Frequency (F) of the bookings/turnover of a customer* - Number of purchases, e.g., in 6 months.
* *Monetary (M)* - The total turnover of a customer: Sum of sales, e.g., in 6 months.

For the analysis, we need to define a ‘analysis date’, which is the day on which we are conducting this analysis which I am taking as the next to last date in data and taking 1 year previous data from the selected date for recency calculation

In [ ]:
# CONVERTING THE InvoiceDate COLUMN INTO DATETIME
uk_df['InvoiceDate'] = pd.to_datetime(uk_df['InvoiceDate'])

In [ ]:
# CREATING AN RFM ANALYSIS DF
analysis_date = uk_df["InvoiceDate"].max() + pd.DateOffset(1)
print("RFM Analysis Date :", analysis_date)

# START DATE FOR OUR ANALYSIS DF
start_date = analysis_date - pd.DateOffset(days = 365)
print("Start Date when taking 1 year data for analysis :", start_date)

In [ ]:
uk_df.info()